## Prompt Engineering

Now that we have a more balanced dataset between Western and Eastern bloc, we will use promt engineering to extract meaningful information from the plot summaries. We will use the `GPT-4o` model to get information about the characters, their archetype and values as well as information on if the film is associated with the Eastern or Western bloc and how they portay their side and the other side.

The following cell initate a client Azure OpenAI API but we don't push the key to the repository.

In [26]:
from openai import AzureOpenAI
import keyring
import pandas as pd
from src.utils.helpers import *

# endpoint = keyring.get_password("AzureOpenAI", "endpoint4omini")
endpoint = keyring.get_password("AzureOpenAI", "endpoint4o")
# key = keyring.get_password("AzureOpenAI", "key4omini")
key = keyring.get_password("AzureOpenAI", "key4o")

model_name = "gpt-4o"

df = pd.read_csv("data/preprocessed/cleaned_merged_movies.csv")

client = AzureOpenAI(azure_endpoint=endpoint,api_version="2024-08-01-preview",api_key=key)

In [4]:
from src.utils.helpers import *

In [37]:
movies_df = df.head(5)
movie_names = ["High School Musical", "From Russia with Love", "The Godfather", "First Blood", "Oshibka rezidenta"]
movies_df =  df[df["title"].isin(movie_names)].copy()
print(df.shape)
movies_df

(32439, 11)


,wikipedia_id,freebase_id,title,languages,countries,genres,keywords,release_date,runtime,plot_summary,year_release_date
8357,872441.0,/m/03kfz0,First Blood,['English'],['United States of America'],"['Psychological thriller', 'Action Thrillers',...","['prison', 'vietnam veteran', 'falsely accused...",1982-10-22,93.0,John Rambo is a former member of an elite Uni...,1982
8887,6446071.0,/m/025twgt,From Russia with Love,"['Romani language', 'English', 'Türkçe', 'Pусс...",['United Kingdom'],"['Glamorized Spy Film', 'Thriller', 'Action', ...","['venice', 'london england', 'terror', 'englan...",1963-10-11,111.0,SPECTRE's expert planner Kronsteen devises a p...,1963
25205,2466773.0,/m/07g1sm,The Godfather,"['English', 'Sicilian', 'Italiano', 'Italian',...",['United States of America'],"['Gangster Film', 'New Hollywood', 'Film adapt...","['italy', 'love at first sight', 'loss of fath...",1972-03-15,177.0,"On the day of his only daughter's wedding, Vit...",1972


We know craft a prompt to get the information we want from the model and use the API to automate the process.

In [ ]:
prompt = "\
You are an expert in movie history and Cold War. You will be given the name of the film, the year and the plot of the movie. You first need to analyse if the movie can be identified to the Eastern or Western bloc during the Cold War. If yes come up with the character or group of character impersonating the Western and Eastern bloc and their values as well as their main archetye.\n\
Your output needs to be parsable comma separated without context (the output needs to start directly), use only keyword and very important use new line character after each of the following:\n\
- Cold War side belonging either Easter, Western or None.\n\
- The character or group of character representing Western bloc with their values and archetype comma separated or None.\n\
- The character or group of character representing Eastern bloc with their values and archetype comma separated or None.\n\
- The Western bloc representation main values and characteristics comma separated or None.\n\
- The Eatern bloc representation main values and characteristics comma separated or None.\n\
- The theme of the movies and keywords.\
"

prompt = "\
You are an expert in movie history and Cold War. You will be given the name of the film, the year and the plot of the movie. You first need to analyse if the movie can be identified to the Eastern or Western bloc during the Cold War. If yes come up with the character or group of character impersonating the Western and Eastern bloc and their values as well as their main archetye.\n\
Your output needs to be parsable comma separated (the output needs to start directly), use only keyword and very important use new line character after each of the following:\n\
- Cold War side belonging either Easter, Western or None.\n\
- The character or group of character representing Western bloc with their values and archetype comma separated or None.\n\
- The character or group of character representing Eastern bloc with their values and archetype comma separated or None.\n\
- The Western bloc representation main values and characteristics comma separated or None.\n\
- The Eatern bloc representation main values and characteristics comma separated or None.\n\
- The theme of the movies and keywords.\n\
Example:\n\
Bloc represented \n\
Name Western Character, First value Western Character, Second value Western Character, etc.\n\
Name Eastern Character, Value1 Eastern Character, Value2 Eastern Character, etc.\n\
First value Western Bloc , Second value Western Bloc, etc.\n\
First value Eastern Bloc, Second value Eastern Bloc, etc.\n\
Theme1, keyword1, keyword2, etc.\n\
If it does not belong to any of the blocs, put None.\n\
Based on the structure above, please provide the following information for the movie:\n\
"


prompt = 

responses = []

for index, row in movies_df.iterrows():
	query = create_query_movie(prompt, row["title"], str(row["year_release_date"]), row["plot_summary"])
	response = client.chat.completions.create(
		model=model_name,
		messages=[
			{"role": "user", "content": query},
		],
	)
	
	responses.append(response.choices[0].message.content)


We now retrieve the data to parse it into new columns in our dataframe.

In [13]:
print(responses)

['Western  \nJohn Rambo, Honor, Survival, War Hero, Tragic Hero  \nWill Teasle, Law Enforcement, Authoritarian, Antagonist  \nNone  \nResilience, Freedom, Individualism, Anti-Establishment  \nBrutality, Oppression, Authoritarianism, Distrust  \nTrauma, Survival, War and its Consequences, keywords: Vietnam War, PTSD, Heroism', 'Western  \nJames Bond, heroism, resourcefulness,  \nTatiana Romanova, loyalty, innocence,  \nRosa Klebb, manipulation, ruthlessness,  \nRed Grant, deception, cold efficiency,  \nBritish intelligence, Western values, individualism, romanticism, spy intrigue,  \nSoviet espionage, control, collectivism, betrayal, treachery,  \nEspionage, betrayal, loyalty, intrigue, cryptography, adventure,', 'None  \nNone  \nNone  \nNone  \nNone  \nCrime, family, power, betrayal, organized crime']


In [35]:
parsed_answers= []

for answer in responses:
	parsed_answers.append(parse_gpt_answer(answer))

parsed_answers

[{'cold_war_side': 'Western',
  'character_western_bloc_representation': ['John Rambo',
   ' survival',
   ' individualism',
   ' war hero'],
  'character_eastern_bloc_representation': ['Art Galt',
   ' cruelty',
   ' oppression',
   ' authority'],
  'western_bloc_values': ['Survival', ' individualism', ' war hero'],
  'eastern_bloc values': ['Cruelty', ' oppression', ' authority'],
  'theme': ['Survival',
   ' post-Vietnam trauma',
   ' individualism',
   ' authority',
   ' struggle',
   ' PTSD']},
 {'cold_war_side': 'Western',
  'character_western_bloc_representation': ['James Bond',
   ' Courage',
   ' Intelligence',
   ' Protection',
   ' Hero'],
  'character_eastern_bloc_representation': ['None'],
  'western_bloc_values': ['Courage', ' Intelligence', ' Protection', ' Hero'],
  'eastern_bloc values': ['None'],
  'theme': ['Espionage',
   ' deception',
   ' Cold War',
   ' cryptography',
   ' SPECTRE',
   ' MI6',
   ' SMERSH',
   ' defection',
   ' Istanbul',
   ' assassination']},


We create 6 new columns `cold_war_side`, `character_western_bloc_representation`, `character_eastern_bloc_representation`, `eastern_bloc_values`, `western_bloc_values` and `theme` to store the information we got from the model.

In [36]:
# create new columns for each key in the parsed answers and assign the values
movies_df = movies_df.assign(**{key: [d[key] for d in parsed_answers] for key in parsed_answers[0].keys()})
movies_df

,wikipedia_id,freebase_id,title,languages,countries,genres,keywords,release_date,runtime,plot_summary,year_release_date,cold_war_side,character_western_bloc_representation,character_eastern_bloc_representation,western_bloc_values,eastern_bloc values,theme
8357,872441.0,/m/03kfz0,First Blood,['English'],['United States of America'],"['Psychological thriller', 'Action Thrillers',...","['prison', 'vietnam veteran', 'falsely accused...",1982-10-22,93.0,John Rambo is a former member of an elite Uni...,1982,Western,"[John Rambo, survival, individualism, war h...","[Art Galt, cruelty, oppression, authority]","[Survival, individualism, war hero]","[Cruelty, oppression, authority]","[Survival, post-Vietnam trauma, individualis..."
8887,6446071.0,/m/025twgt,From Russia with Love,"['Romani language', 'English', 'Türkçe', 'Pусс...",['United Kingdom'],"['Glamorized Spy Film', 'Thriller', 'Action', ...","['venice', 'london england', 'terror', 'englan...",1963-10-11,111.0,SPECTRE's expert planner Kronsteen devises a p...,1963,Western,"[James Bond, Courage, Intelligence, Protect...",[None],"[Courage, Intelligence, Protection, Hero]",[None],"[Espionage, deception, Cold War, cryptograp..."
25205,2466773.0,/m/07g1sm,The Godfather,"['English', 'Sicilian', 'Italiano', 'Italian',...",['United States of America'],"['Gangster Film', 'New Hollywood', 'Film adapt...","['italy', 'love at first sight', 'loss of fath...",1972-03-15,177.0,"On the day of his only daughter's wedding, Vit...",1972,None,[None],[None],[None],[None],"[Crime, Family, Power, Loyalty, Betrayal]"


This is a first analysis that shows that the model is able to extract meaningful information from the plot summaries. 

Films that are not associated with the Cold Ware are well detected which is reassuring.

We will use this information to cluster the movies in the next milestone.